## Welcome to the MDC Trends Detector Tool

A few notes before we begin the analysis:

- *Cells are labeled at the top to provided order to the tool.*

- *To run a cell: Click into a cell, click "Run" above or "ctrl + Enter"*

- *To learn more shortcuts, press "esc" and H to see a list of shortcuts*

- *Analysis (daily and history) are intentionally kept separate to avoid confusion and add more versatility to the tool, before starting a separate analysis click on the circular arrow above*

1. To begin with a clean slate click on the "Circular Arrow" above to restart the kernel (session)

2. Run cell #1 to load the necessary libraries for this tool

In [5]:
# 1
# Importing libraries to the notebook
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib
import datetime
import time

#%matplotlib qt

In [6]:
# 2
# Importing the data into the notebook
Datatobeanalyzed = r"/Users/kanu/Desktop/MHIRJ/sepData.csv" # Input file path here
MDCMessagesURL = "MDCMessagesInputs.csv"
TopMessagesURL = "TopMessagesSheet.csv"

In [8]:
# 3
# Constant to both analysis
MDCdataDF = pd.read_csv(Datatobeanalyzed, encoding="utf8")
MDCdataDF["DateAndTime"] = pd.to_datetime(MDCdataDF["DateAndTime"]) # formatting for date
MDCdataDF["Flight Leg No"].fillna(value= 0.0, inplace= True) # Null values preprocessing - if 0 = Currentflightphase
MDCdataDF["Flight Phase"].fillna(False, inplace= True) # NuCell values preprocessing for currentflightphase
MDCdataDF["Intermittent"].fillna(value= 0.0, inplace= True) # Null values preprocessing for currentflightphase
MDCdataDF["Intermittent"].replace(to_replace= ">", value= "9", inplace=True) # > represents greater than 8 Intermittent values
MDCdataDF["Intermittent"] = MDCdataDF["Intermittent"].astype(int) # cast type to int

MDCdataDF["Aircraft"] = MDCdataDF["Aircraft"].str.replace('AC', '')
MDCdataDF.fillna(value= " ", inplace= True) # replacing all REMAINING null values to a blank string
MDCdataDF.sort_values(by= "DateAndTime", ascending= False, inplace= True, ignore_index= True)

AircraftTailPairDF = MDCdataDF[["Aircraft", "Tail#"]].drop_duplicates(ignore_index= True) # unique pairs of AC SN and Tail# for use in analysis
AircraftTailPairDF.columns = ["AC SN","Tail#"] # re naming the columns to match History/Daily analysis output

DatesinData = MDCdataDF["DateAndTime"].dt.date.unique() # these are the dates in the data in Datetime format. 
NumberofDays = len(MDCdataDF["DateAndTime"].dt.date.unique()) # to pass into Daily analysis number of days in data
latestDay = str(MDCdataDF.loc[0, "DateAndTime"].date()) # to pass into history analysis MDCdataDF["DateAndTime"].sort_values().iloc[-1]

MDCMessagesDF = pd.read_csv(MDCMessagesURL, encoding="utf8") # bring messages and inputs into a Dataframe
TopMessagesDF = pd.read_csv(TopMessagesURL) # bring messages and inputs into a Dataframe
TopMessagesArray = TopMessagesDF.to_numpy() # converting to numpy to work with arrays

# this list can be updated to include other messages
listofJamMessages = ["B1-309178","B1-309179","B1-309180","B1-060044","B1-060045","B1-007973",
                     "B1-060017","B1-006551","B1-240885","B1-006552","B1-006553","B1-006554",
                     "B1-006555","B1-007798","B1-007772","B1-240938","B1-007925","B1-007905",
                     "B1-007927","B1-007915","B1-007926","B1-007910","B1-007928","B1-007920"]

# function used later on to highlight the messages above if flagged
def highlightJams(Outputtable, listofmessages= listofJamMessages, column= "B1-Equation"):
    ''' highlights the rows where a Jam message was flagged on the report. 
    Note: This color will also print on to_excel 
    Outputtable: Either Historyreport or dailyreport
    listofmessages: List of jam messages, but could also add other types of messages to flag
    Column: column to check
    
    you can use this link to change the color to highlight
    https://matplotlib.org/stable/gallery/color/named_colors.html'''
    
    is_jam = pd.Series(data=False, index= Outputtable.index)
    is_jam[column] = Outputtable.loc[column] in listofmessages
    return ['background-color: #f08080' if is_jam.any() else '' for v in is_jam]

#function used later on to display the messages in the same Flight Leg as the Jam flags raised
def flagsinreport(OutputTable, Aircraft, listofmessages= listofJamMessages):
    ''' display the messages in the same Flight Leg as the Jam flags
        OutputTable: Either HistoryReport or Dailyreport
        Listofmessages: Jam Messages
        Return: Dataframe indexed with AC SN and Flight Leg #
    '''
    datatofilter = MDCdataDF.copy(deep= True)
    
    isin = OutputTable["B1-Equation"].isin(listofmessages)
    filter1 = OutputTable[isin][["AC SN", "B1-Equation"]]
    listoftuplesACID = list(zip(filter1["AC SN"], filter1["B1-Equation"]))
    
    datatofilter2 = datatofilter.set_index(["Aircraft", "Equation ID"]).sort_index().loc[pd.IndexSlice[listoftuplesACID], :].reset_index()
    listoftuplesACFL = list(zip(datatofilter2["Aircraft"], datatofilter2["Flight Leg No"]))
    
    datatofilter3 = datatofilter.set_index(["Aircraft", "Flight Leg No"]).sort_index()
    FinalDF = datatofilter3.loc[pd.IndexSlice[listoftuplesACFL], :]
    return FinalDF.loc[Aircraft]

def separate_data(data, date):
    '''Takes data as a dataframe, along with a date to slice the larger data to only include the data in that date'''
    
    DailyDataDF = data.loc[date]
    return DailyDataDF

def LongestConseq(unique_arr, days_legs):
    '''
    Finds the maximum consecutive number of days or legs in the passed array. 
    Passed array has to be sorted in descending order for the algorithm to work

    Input:
    unique_arr: A sorted array/list (descending) in datetime format to operate on
    days_Legs: "days" or "legs" user choice 

    Output:
    ans: Maximum number of consecutive days or legs in the array
    '''

    ans = 0
    count = 0

    if days_legs == "days":
        # Find the maximum length
        # by traversing the array
        for i in range(len(unique_arr)):
            if (i > 0 and unique_arr[i] == unique_arr[i - 1] - datetime.timedelta(1)):
                count += 1

            # Reset the count
            else:
                count = 1

            # Update the maximum
            ans = max(ans, count)

    elif days_legs == "legs":
        # Find the maximum length
        # by traversing the array
        for i in range(len(unique_arr)):
            if (i > 0 and int(unique_arr[i]) == int(unique_arr[i - 1] -1)):
                count += 1

            # Reset the count
            else:
                count = 1

            # Update the maximum
            ans = max(ans, count)
    else:
        raise NameError("Only 'days' or 'legs' can be passed into days_legs arg")

    return ans    

def check_2in5(dates):
    '''Returns true if the list of days passed has 2 entries within 5 days'''

    # Iterate through days
    for i in range(len(dates)):

        if i == 0:pass

        # If day is less than 5 days before the previous day return true
        elif dates[i] > dates[i-1] - datetime.timedelta(5): return True

    return False

def b1Format ():
    code = input("Enter the B1-code you want to add: ").upper() # input the B1 Message code you want to study
    if "B" not in code: code = "B1-" + code
    return code

def highlight_delta(table_highlight, delta_list, Jam_list= listofJamMessages):
    '''highlights delta of two history reports'''
    # check what in the new report exists in the prev. which of tuple(AC SN, B1) exists in the true list
    is_color = (table_highlight["AC SN"], table_highlight["B1-Equation"]) in delta_list
    
    # already existed in prev report and is not jam
    if delta_list == True_list and table_highlight["B1-Equation"] not in Jam_list:
        # light orange
        return ['background-color: #fde9d9' if is_color else '' for v in table_highlight]
    # didnt exist in prev report and is not jam
    elif delta_list == False_list and table_highlight["B1-Equation"] not in Jam_list:
        # dark orange
        return ['background-color: #fabf8f' if is_color else '' for v in table_highlight]
    # already existed in prev report and is jam
    elif delta_list == True_list and table_highlight["B1-Equation"] in Jam_list:
        # light red
        return ['background-color: #f08080' if is_color else '' for v in table_highlight]
    # didnt exist in prev report and is jam
    elif delta_list == False_list and table_highlight["B1-Equation"] in Jam_list:
        # dark red
        return ['background-color: #ff342e' if is_color else '' for v in table_highlight]

def create_delta_lists(prev_history, curr_history):
    '''create True and False lists for the Delta report'''
    # making tuples of the combos of AC SN and B1-eqn
    comb_prev = list(zip(prev_history["AC SN"], prev_history["B1-Equation"]))
    comb_curr = list(zip(curr_history["AC SN"], curr_history["B1-Equation"]))

    # create a list for flags still on going (true_list) and new flags (false_list)
    True_list = []
    False_list = []
    for i in range(len(comb_curr)):
        if comb_curr[i] in comb_prev:
            True_list.append(comb_curr[i])
        elif comb_curr[i] not in comb_prev:
            False_list.append(comb_curr[i])
    return True_list, False_list

# Chart A

Shows the magnitude of messages with different colours for ATA for each AC

In [8]:
# https://towardsdatascience.com/stacked-bar-charts-with-pythons-matplotlib-f4020e4eb4a7
# https://stackoverflow.com/questions/44309507/stacked-bar-plot-using-matplotlib

Topvalues2 = 15
# groups the data by Aircraft and Main ATA, produces a count of values in each ata by counting entries in Equation ID
# chartADF = MDCdataDF[["Aircraft","ATA Main", "Equation ID"]]
# chartADF.columns = ["AC SN", "ATA Main", "Equation ID"]
chartADF = pd.merge(left = MDCdataDF[["Aircraft","ATA Main", "Equation ID"]], right = AircraftTailPairDF, left_on="Aircraft", right_on="AC SN")
chartADF["Aircraft"] = chartADF["Aircraft"] + " / " + chartADF["Tail#"]
chartADF.drop(labels = ["AC SN", "Tail#"], axis = 1, inplace = True)
MessageCountbyAircraftATA = chartADF.groupby(["Aircraft","ATA Main"]).count()

# transpose the indexes. where the ATA label becomes the column and the aircraft is row. counts are middle
TransposedMessageCountbyAircraftATA = MessageCountbyAircraftATA["Equation ID"].unstack()

# fill Null values with 0
TransposedMessageCountbyAircraftATA.fillna(value= 0, inplace= True)

# sum all the counts by row, plus create a new column called sum
TransposedMessageCountbyAircraftATA["Sum"] = TransposedMessageCountbyAircraftATA.sum(axis=1)

# sort the dataframe by the values of sum, and from the topvalues2 the user chooses
TransposedMessageCountbyAircraftATA = TransposedMessageCountbyAircraftATA.sort_values("Sum").tail(Topvalues2)

# create a final dataframe for plotting without the new column created before
TransposedMessageCountbyAircraftATAfinalPLOT = TransposedMessageCountbyAircraftATA.drop(["Sum"], axis=1)

totals = TransposedMessageCountbyAircraftATA["Sum"]
#image settings
ax8 = TransposedMessageCountbyAircraftATAfinalPLOT.plot(kind='barh', stacked=True, figsize=(16, 9))
ax8.set_ylabel('Aircraft Serial Number')
ax8.set_title('Magnitude of messages in data')
ax8.grid(b= True, which= "both", axis= "x", alpha= 0.3)
rects8 = ax8.containers[-1] 


# here to add column labeling
for i, total in enumerate(totals):
    ax8.text(totals[i], rects8[i].get_y() +0.15 , round(total), ha='left')
    
plt.show()

# Chart B

User picks an AC and gets the breakdown per ATA

In [11]:
Aircrafttostudy5 = input("Select the aircraft to analyze: ")

Select the aircraft to analyze: 10251


In [12]:
# groups the data by Aircraft and Main ATA, produces a count of values in each ata by counting entries in Equation ID
MessageCountbyAircraftATA2 = MDCdataDF[["Aircraft","ATA Main", "Equation ID"]].groupby(["Aircraft","ATA Main"]).count()

# transpose the indexes. where the ATA label becomes the column and the aircraft is row. counts are middle
TransposedMessageCountbyAircraftATA2 = MessageCountbyAircraftATA2["Equation ID"].unstack()

Labels8 = TransposedMessageCountbyAircraftATA2.loc[Aircrafttostudy5].dropna().sort_values(ascending= False).index
Counts8 = TransposedMessageCountbyAircraftATA2.loc[Aircrafttostudy5].dropna().sort_values(ascending= False)

#Image settings
spacing8 = np.arange(start= 0, stop= len(Labels8)*2, step= 2) # label locations
width = 0.9 # bar height
fig9, ax9 = plt.subplots() # objects for figure and axes, figsize controls the size of the output window
rects9 = ax9.bar(spacing8, Counts8, label= "Count of message:")

ax9.set_xlabel('ATA')
ax9.set_ylabel("Count")
ax9.set_xticks(spacing8)
ax9.set_xticklabels(Labels8)
tail = AircraftTailPairDF.loc[AircraftTailPairDF["AC SN"] == Aircrafttostudy5].iloc[0].at["Tail#"]
ax9.set_title('ATA Breakdown for Aircraft: ' + Aircrafttostudy5 + " / " + tail)
ax9.grid(b= True, alpha= 0.3)

def autolabel(rects): # https://matplotlib.org/examples/api/barchart_demo.html
    """
    Attach a text label above each bar displaying its height
    """
    for rect in rects:
        height = rect.get_height()
        ax9.text(rect.get_x() + rect.get_width()/2., 1.05*height,
                '%d' % int(height),
                ha='center', va='bottom') #hor alignment

autolabel(rects9) 
fig9.tight_layout()
plt.show()


# Daily analysis
The first type of analysis is the daily analysis where it only takes the latest day in the data to be analyzed. 

- Fill out the user inputs below to set the flags for the analysis and run cell #4

- Run cell #5 to perform the Daily Analysis

**If performing History Analysis skip to Cell #8**

* Note: if you restarted the kernel, remember to go back to cell #1 to initialize the libraries

## User Inputs:

In [ ]:
# 4
# User Inputs for Daily Analysis

CurrentFlightPhase = input("Include or Exclude Current Flight Phase: ")
# 1 or 0, 1 includes current phase, 0 does not include current phase
if CurrentFlightPhase.lower() == "include":
    CurrentFlightPhaseEnabled = 1
elif CurrentFlightPhase.lower() == "exclude":
    CurrentFlightPhaseEnabled = 0
else:
    print("WARNING: only include and exclude are allowed as inputs for this value")
# flag for Total number of occurrences
MaxAllowedOccurrences = int(input("Number for total number of daily occurrences allowed: "))
# flag for consecutive legs
MaxAllowedConsecLegs = int(input("Number for maximum daily consecutive legs allowed: "))
# flag for intermittent values
MaxAllowedIntermittent = int(input("Number for maximum daily allowed intermittent value: "))
if MaxAllowedIntermittent > 9:
    print("WARNING: 9 is the maximum value for intermittence")
elif MaxAllowedIntermittent == 9:
    print("9 represents the '>' symbol in raw data")


## Code for Daily Analysis:

In [ ]:
# 5

currentRow = 0
MAINtable_array_temp = np.empty((1,19),object) # 18 for the date 
MAINtable_array = []

AnalysisDF = MDCdataDF.set_index("DateAndTime") # since dateandtime was moved to the index of the DF, the column values change from the original MDCdataDF

for i in range(0, NumberofDays):
    
    daytopass = str(DatesinData[i])
    #define array to analyze
    DailyanalysisDF = separate_data(AnalysisDF, daytopass)
    
    # total occurrences
    if CurrentFlightPhase.lower() == "include":
        # include the current flight legs
        selection = DailyanalysisDF[["Equation ID", "Aircraft"]]
        total_occ_DF = selection.value_counts().unstack()

        # selecting only the necessary data for - consecutive legs
        legs_selection = DailyanalysisDF[["Aircraft", "Equation ID", "Flight Leg No"]].sort_values(by= ["Flight Leg No"], ascending= False).copy()
        legs_selection.set_index(["Aircraft", "Equation ID"], inplace= True)
        legs_selection.sort_index(inplace= True)

        # selecting only the necessary data for - intermittent
        Intermittent_selection = DailyanalysisDF[["Aircraft", "Equation ID", "Intermittent"]].copy()
        Intermittent_selection.set_index(["Aircraft", "Equation ID"], inplace= True)
        Intermittent_selection.sort_index(inplace= True)

    elif CurrentFlightPhase.lower() == "exclude":
        # exclude the current flight legs
        selection = DailyanalysisDF[["Equation ID", "Aircraft", "Flight Leg No"]][DailyanalysisDF["Flight Leg No"] != 0].copy()
        selection.drop(["Flight Leg No"], inplace= True, axis= 1)
        total_occ_DF = selection.value_counts().unstack()

        # selecting only the necessary data for - consecutive legs
        legs_selection = DailyanalysisDF[["Aircraft", "Equation ID", "Flight Leg No"]][DailyanalysisDF["Flight Leg No"] != 0].sort_values(by= ["Flight Leg No"], ascending= False).copy()
        legs_selection.set_index(["Aircraft", "Equation ID"], inplace= True)
        legs_selection.sort_index(inplace= True)

        # selecting only the necessary data for - intermittent
        Intermittent_selection = DailyanalysisDF[["Aircraft", "Equation ID", "Intermittent", "Flight Leg No"]][DailyanalysisDF["Flight Leg No"] != 0].copy()
        Intermittent_selection.drop(["Flight Leg No"], inplace= True, axis= 1)
        Intermittent_selection.set_index(["Aircraft", "Equation ID"], inplace= True)
        Intermittent_selection.sort_index(inplace= True)

    # obtain location of nonempty values
    b1message, aircraft = np.where(pd.notnull(total_occ_DF))
    # obtains the address to the values to be referenced later
    notEmptyLabelPairs = np.column_stack([total_occ_DF.columns[aircraft],total_occ_DF.index[b1message]])

    # creating a dataframe with a similar size to total occurrences df
    consec_legs = pd.DataFrame().reindex_like(total_occ_DF)

    # creating a dataframe with a similar size to total occurrences df
    max_intermittent = pd.DataFrame().reindex_like(total_occ_DF)

    # creating a dataframe with a similar size to total occurrences df
    flags_array = pd.DataFrame().reindex_like(total_occ_DF)

    # fill null values with 0
    consec_legs.fillna(value= 0.0, inplace = True)
    max_intermittent.fillna(value= 0.0, inplace= True)
    total_occ_DF.fillna(value= 0.0, inplace= True)
    flags_array.fillna(value= "", inplace= True)

    # Creating flags lists
    flags_jams = MDCMessagesDF.loc[(MDCMessagesDF["Occurrence Flag"] == 1) & (MDCMessagesDF["Days Count"] == 0)]["Equation ID"]

    # go through AC/eqnID combinations for analysis
    for i in range(len(notEmptyLabelPairs)):

        # pick AC and eqn ID combo
        aircraft = notEmptyLabelPairs[i, 0]
        equation = notEmptyLabelPairs[i, 1]

        # set up legs selection for analysis
        legs = legs_selection.loc[aircraft, equation]
        if len(legs) > 1:
            legs = legs["Flight Leg No"].unique()
            consec_legs.at[equation, aircraft] = LongestConseq(unique_arr= legs, days_legs= "legs")
            
            if (legs > 32600).any():
                legs_flag = True
            else:
                legs_flag = False
                
        elif len(legs) == 1:
            legs = legs["Flight Leg No"].item()
            consec_legs.at[equation, aircraft] = 0
            
            if legs > 32600:
                legs_flag = True
            else:
                legs_flag = False

        # set up intermitt selection for analysis
        intermitt = Intermittent_selection.loc[aircraft, equation]

        # run
        
        if len(intermitt) > 1:
            max_intermittent.at[equation, aircraft] = max(intermitt["Intermittent"])
        elif len(intermitt) == 1:
            max_intermittent.at[equation, aircraft] = intermitt["Intermittent"].item()
            
        if total_occ_DF.at[equation, aircraft] >= MaxAllowedOccurrences \
        or consec_legs.at[equation, aircraft] >= MaxAllowedConsecLegs \
        or max_intermittent.at[equation, aircraft] >= MaxAllowedIntermittent \
        or legs_flag \
        or (flags_jams == equation).any():

            if total_occ_DF.at[equation, aircraft] >= MaxAllowedOccurrences:
                flags_array.at[equation, aircraft] = flags_array.at[equation, aircraft] + "Total occurrences exceeded " + str(MaxAllowedOccurrences) + " occurrences. "

            if consec_legs.at[equation, aircraft] >= MaxAllowedConsecLegs:
                flags_array.at[equation, aircraft] = flags_array.at[equation, aircraft] + "Maximum consecutive flight legs exceeded " + str(MaxAllowedConsecLegs) + " flight legs. "

            if max_intermittent.at[equation, aircraft] >= MaxAllowedIntermittent:
                flags_array.at[equation, aircraft] = flags_array.at[equation, aircraft] + "Maximum intermittent occurrences for one flight leg exceeded " + str(MaxAllowedIntermittent) + " occurrences. "

            if legs_flag:
                flags_array.at[equation, aircraft] = flags_array.at[equation, aircraft] + "Flight legs above 32600. "

            if (flags_jams == equation).any():
                flags_array.at[equation, aircraft] = flags_array.at[equation, aircraft] + str(equation) + " occurred at least once. "

            #populating the final array (Table)
            MAINtable_array_temp[0,0] = daytopass
            MAINtable_array_temp[0,1] = aircraft # SN
            MAINtable_array_temp[0,2] = MDCMessagesDF["EICAS"][MDCMessagesDF["Equation ID"] == equation].item() #EICAS
            MAINtable_array_temp[0,3] = MDCMessagesDF["Message"][MDCMessagesDF["Equation ID"] == equation].item() #Messge
            MAINtable_array_temp[0,4] = MDCMessagesDF["LRU"][MDCMessagesDF["Equation ID"] == equation].item() #LRU
            MAINtable_array_temp[0,5] = MDCMessagesDF["ATA"][MDCMessagesDF["Equation ID"] == equation].item() #ATA
            MAINtable_array_temp[0,6] = equation #Eqn ID
            MAINtable_array_temp[0,7] = MDCMessagesDF["Message Type"][MDCMessagesDF["Equation ID"] == equation].item()
            MAINtable_array_temp[0,8] = MDCMessagesDF["Equation Description"][MDCMessagesDF["Equation ID"] == equation].item()
            MAINtable_array_temp[0,9] = total_occ_DF.at[equation, aircraft]
            
            MAINtable_array_temp[0,10] = consec_legs.at[equation, aircraft]
            MAINtable_array_temp[0,11] = max_intermittent.at[equation, aircraft]
            MAINtable_array_temp[0,12] = str(flags_array.at[equation, aircraft])

            #if the input is empty set the priority to 4
            if MDCMessagesDF["Priority "][MDCMessagesDF["Equation ID"] == equation].item() == 0:
                MAINtable_array_temp[0,13] = 4
            else:
                MAINtable_array_temp[0,13] = MDCMessagesDF["Priority "][MDCMessagesDF["Equation ID"] == equation].item()

            #For B1-006424 & B1-006430 Could MDC Trend tool assign Priority 3 if logged on A/C below 10340, 15317. Priority 1 if logged on 10340, 15317, 19001 and up
            if equation == "B1-006424" or equation == "B1-006430":
                if int(aircraft) <= 10340 and int(aircraft) > 10000:
                    MAINtable_array_temp[0,13] = 3
                elif int(aircraft) > 10340 and int(aircraft) < 11000:
                    MAINtable_array_temp[0,13] = 1
                elif int(aircraft) <= 15317 and int(aircraft) > 15000:
                    MAINtable_array_temp[0,13] = 3
                elif int(aircraft) > 15317 and int(aircraft) < 16000:
                    MAINtable_array_temp[0,13] = 1
                elif int(aircraft) >= 19001 and int(aircraft) < 20000:
                    MAINtable_array_temp[0,13] = 1

            #check the content of MHIRJ ISE recommendation and add to array    
            if MDCMessagesDF["MEL or No-Dispatch"][MDCMessagesDF["Equation ID"] == equation].item() == "0":
                MAINtable_array_temp[0,15] = ""
            else:
                MAINtable_array_temp[0,15] = MDCMessagesDF["MEL or No-Dispatch"][MDCMessagesDF["Equation ID"] == equation].item()

            #check content of "MHIRJ Input"
            if MDCMessagesDF["MHIRJ ISE inputs"][MDCMessagesDF["Equation ID"] == equation].item() == "0":
                MAINtable_array_temp[0,16] = ""
            else:
                MAINtable_array_temp[0,16] = MDCMessagesDF["MHIRJ ISE inputs"][MDCMessagesDF["Equation ID"] == equation].item()
        
            #check the content of MHIRJ ISE recommendation and add to array    
            if MDCMessagesDF["MHIRJ ISE Recommended Action"][MDCMessagesDF["Equation ID"] == equation].item() == "0":
                MAINtable_array_temp[0,17] = ""
            else:
                MAINtable_array_temp[0,17] = MDCMessagesDF["MHIRJ ISE Recommended Action"][MDCMessagesDF["Equation ID"] == equation].item()

            #check content of "additional"
            if MDCMessagesDF["Additional Comments"][MDCMessagesDF["Equation ID"] == equation].item() == "0":
                MAINtable_array_temp[0,18] = ""
            else:
                MAINtable_array_temp[0,18] = MDCMessagesDF["Additional Comments"][MDCMessagesDF["Equation ID"] == equation].item()


            #Check for the equation in the Top Messages sheet
            TopCounter = 0
            Top_LastRow = TopMessagesArray.shape[0]
            while TopCounter < Top_LastRow:

                #Look for the flagged equation in the Top Messages Sheet
                if equation == TopMessagesArray[TopCounter,4]:

                    #Found the equation in the Top Messages Sheet. Put the information in the report
                    MAINtable_array_temp[0,14] = "Known Nuissance: " + str(TopMessagesArray[TopCounter, 13]) \
                    + " / In-Service Document: " + str(TopMessagesArray[TopCounter,11]) \
                    + " / FIM Task: " + str(TopMessagesArray[TopCounter,10]) \
                    + " / Remarks: " + str(TopMessagesArray[TopCounter,14])

                    #Not need to keep looking
                    TopCounter = TopMessagesArray.shape[0]

                else:
                    #Not equal, go to next equation
                    MAINtable_array_temp[0,14] = ""
                    TopCounter += 1
            # End while

            if currentRow == 0:
                MAINtable_array = np.array(MAINtable_array_temp)      
            else:
                MAINtable_array = np.append(MAINtable_array,MAINtable_array_temp,axis=0)
            #End if Build MAINtable_array

            #Move to next Row on Main page for next flag
            currentRow = currentRow + 1

TitlesArrayDaily = ["Date", "AC SN", "EICAS Message", "MDC Message", "LRU", "ATA", "B1-Equation", "Type",
           "Equation Description", "Total Occurrences", "Consecutive FL",
          "Intermittent", "Reason(s) for flag", "Priority", "Known Top Message - Recommended Documents", "MEL or No-Dispatch", 
              "MHIRJ Input", "MHIRJ Recommendation", "Additional Comments"]

# Converts the Numpy Array to Dataframe to manipulate
#pd.set_option('display.max_rows', None)
# Main table

OutputTableDaily = pd.DataFrame(data= MAINtable_array, columns= TitlesArrayDaily).fillna(" ").sort_values(by= ["Date", "Type", "Priority"])
OutputTableDaily = OutputTableDaily.merge(AircraftTailPairDF, on= "AC SN") # Tail # added
OutputTableDaily = OutputTableDaily[["Tail#", "Date", "AC SN", "EICAS Message", "MDC Message", "LRU", "ATA", "B1-Equation", "Type",
               "Equation Description", "Total Occurrences", "Consecutive FL",
              "Intermittent", "Reason(s) for flag", "Priority", "Known Top Message - Recommended Documents", "MEL or No-Dispatch",
              "MHIRJ Input", "MHIRJ Recommendation", "Additional Comments"]] # Tail# added to output table which means that column order has to be re ordered


Run cell #6 to choose if you want to print the report to excel, and see a preview of the report


In [ ]:
# 6
# to toggle the output in this cell (or any cell) press "esc + o" - find more shortcuts using "esc + h"
# Check if user desires excel output

# In order to properly show the color, the "style" method seen below will change the Dataframe data type to a "Styler"
# data type. which will not allow other actions to work (i.e. drop, apply, etc) since those methods dont exist for Styler
# data type. a copy of the output table is created to display and rewrite that table everytime the cell is run. 
copy_OutputTableDaily = OutputTableDaily.copy(deep= True)
copy_OutputTableDaily = copy_OutputTableDaily.style.apply(highlightJams, axis=1)


excelcheck  = input("Do you want to print this report to excel? yes/no: ")
if excelcheck.lower() != "yes" and excelcheck.lower() != "no":
    print("Only yes or no allowed")
elif excelcheck.lower() == "no":
    display(copy_OutputTableDaily)
elif excelcheck.lower() == "yes":
    copy_OutputTableDaily.to_excel(input("Input the desired filename, '.xlsx' is added automatically: ") + ".xlsx", index= False, header= True)
    display(copy_OutputTableDaily)

# If there is a Jam related flag, the supporting info will show up here

Run the cell below to obtain the other messages posted on the same flight leg as the Jam Message.
Input the Aircraft SN: 1XXXX to obtain the messages related to the Jam.

Example of process would be:
- Run Daily Report
- Look at the highlighted flags, choose one of those Aircraft SN and copy
- Paste it into the input box below
- You will obtain the list of messages that also occurred on the same flight leg
- You can also choose to print that table to excel in the next cell


In [ ]:
ACchosen = input("From your analysis, what Aircraft SN do you want to look at?: ")

RelatedtoFlagged = flagsinreport(OutputTable= OutputTableDaily, Aircraft= ACchosen)
RelatedtoFlagged

In [ ]:
RelatedtoFlagged.to_excel("") # write the desired file name in the quotes

this is here in case you want to look at the raw data by aircraft.

Select the aircraft you want by typing into the prompt 

In [ ]:
# 7

MDCdataDFbyAC = MDCdataDF.set_index("Aircraft")
MDCdataDFbyAC.loc[input("Input the aircraft number: 1XXXX  ")]

## History Analysis

The second type of analysis is the History analysis where it only takes the entire dataset to detect trends over time. 

- Fill out the user inputs below to set the flags for the analysis and run cell #8

- Run cell #9 to perform the History Analysis

If performing Daily Analysis go back to Cell #4

* Note: if you restarted the kernel, remember to go back to cell #1 to initialize the libraries

## User Inputs:

In [12]:
# 8
# User Inputs for History Analysis

CurrentFlightPhase = input("Include or Exclude Current Flight Phase: ")
# 1 or 0, 1 includes current phase, 0 does not include current phase
if CurrentFlightPhase.lower() == "include":
    CurrentFlightPhaseEnabled = 1
elif CurrentFlightPhase.lower() == "exclude":
    CurrentFlightPhaseEnabled = 0
else:
    raise NameError("only 'include' and 'exclude' are allowed as inputs for this value")
# flag for Total number of occurrences
MaxAllowedOccurrences = int(input("Number for total number of occurrences allowed: "))
# flag for consecutive days
MaxAllowedConsecDays =  int(input("Number for maximum consecutive days allowed: "))
# flag for consecutive legs
MaxAllowedConsecLegs = int(input("Number for maximum consecutive legs allowed: "))
# flag for intermittent values
MaxAllowedIntermittent = int(input("Number for maximum allowed intermittent value: "))
if MaxAllowedIntermittent > 9:
    print("WARNING: 9 is the maximum value for intermittence")
elif MaxAllowedIntermittent == 9:
    print("9 represents the '>' symbol in raw data")


In [13]:
# 9

# total occurrences
if CurrentFlightPhase.lower() == "include":
    # include the current flight legs
    selection = MDCdataDF[["Equation ID", "Aircraft"]]
    total_occ_DF = selection.value_counts().unstack()
    
    # selecting only the necessary data for - consecutive days
    dates_selection = MDCdataDF[["Aircraft", "Equation ID", "DateAndTime"]].copy()
    dates_selection.set_index(["Aircraft", "Equation ID"], inplace= True)
    dates_selection.sort_index(inplace= True)
    
    # selecting only the necessary data for - consecutive legs
    legs_selection = MDCdataDF[["Aircraft", "Equation ID", "Flight Leg No"]].sort_values(by= ["Flight Leg No"], ascending= False).copy()
    legs_selection.set_index(["Aircraft", "Equation ID"], inplace= True)
    legs_selection.sort_index(inplace= True)
    
    # selecting only the necessary data for - intermittent
    Intermittent_selection = MDCdataDF[["Aircraft", "Equation ID", "Intermittent"]].copy()
    Intermittent_selection.set_index(["Aircraft", "Equation ID"], inplace= True)
    Intermittent_selection.sort_index(inplace= True)
    
elif CurrentFlightPhase.lower() == "exclude":
    # exclude the current flight legs
    selection = MDCdataDF[["Equation ID", "Aircraft", "Flight Leg No"]][MDCdataDF["Flight Leg No"] != 0].copy()
    selection.drop(["Flight Leg No"], inplace= True, axis= 1)
    total_occ_DF = selection.value_counts().unstack()
    
    # selecting only the necessary data for - consecutive days
    dates_selection = MDCdataDF[["Aircraft", "Equation ID", "DateAndTime", "Flight Leg No"]][MDCdataDF["Flight Leg No"] != 0].copy()
    dates_selection.drop(["Flight Leg No"], inplace= True, axis= 1)
    dates_selection.set_index(["Aircraft", "Equation ID"], inplace= True)
    dates_selection.sort_index(inplace= True)
    
    # selecting only the necessary data for - consecutive legs
    legs_selection = MDCdataDF[["Aircraft", "Equation ID", "Flight Leg No"]][MDCdataDF["Flight Leg No"] != 0].sort_values(by= ["Flight Leg No"], ascending= False).copy()
    legs_selection.set_index(["Aircraft", "Equation ID"], inplace= True)
    legs_selection.sort_index(inplace= True)
    
    # selecting only the necessary data for - intermittent
    Intermittent_selection = MDCdataDF[["Aircraft", "Equation ID", "Intermittent", "Flight Leg No"]][MDCdataDF["Flight Leg No"] != 0].copy()
    Intermittent_selection.drop(["Flight Leg No"], inplace= True, axis= 1)
    Intermittent_selection.set_index(["Aircraft", "Equation ID"], inplace= True)
    Intermittent_selection.sort_index(inplace= True)
    
# obtain location of nonempty values
b1message, aircraft = np.where(pd.notnull(total_occ_DF))
# obtains the address to the values to be referenced later
notEmptyLabelPairs = np.column_stack([total_occ_DF.columns[aircraft],total_occ_DF.index[b1message]])

# creating a dataframe with a similar size to total occurrences df
consec_days = pd.DataFrame().reindex_like(total_occ_DF)

# creating a dataframe with a similar size to total occurrences df
consec_legs = pd.DataFrame().reindex_like(total_occ_DF)

# creating a dataframe with a similar size to total occurrences df
max_intermittent = pd.DataFrame().reindex_like(total_occ_DF)

# creating a dataframe with a similar size to total occurrences df
flags_array = pd.DataFrame().reindex_like(total_occ_DF)

# fill null values with 0
consec_legs.fillna(value= 0.0, inplace = True)
consec_days.fillna(value= 0.0, inplace= True)
max_intermittent.fillna(value= 0.0, inplace= True)
total_occ_DF.fillna(value= 0.0, inplace= True)
flags_array.fillna(value= "", inplace= True)

# Creating flags lists
flags_jams = MDCMessagesDF.loc[(MDCMessagesDF["Occurrence Flag"] == 1) & (MDCMessagesDF["Days Count"] == 0)]["Equation ID"]
flags_2in5 = MDCMessagesDF.loc[(MDCMessagesDF["Occurrence Flag"] == 2) & (MDCMessagesDF["Days Count"] == 5)]["Equation ID"]

# create main table array
MAINtable_array_temp = np.empty((1,21),object) # 21 = # of columns
currentRow = 0
MAINtable_array = []

# go through AC/eqnID combinations for analysis
for i in range(len(notEmptyLabelPairs)):
    
    # pick AC and eqn ID combo
    aircraft = notEmptyLabelPairs[i, 0]
    equation = notEmptyLabelPairs[i, 1]
    
    # set up legs selection for analysis
    legs = legs_selection.loc[aircraft, equation]
    legs = legs["Flight Leg No"].unique()
    
    # set up dates selection for analysis
    dates = dates_selection.loc[aircraft, equation]
    dates = dates["DateAndTime"].dt.date.unique()
    
    # set up intermitt selection for analysis
    intermitt = Intermittent_selection.loc[aircraft, equation].Intermittent
    
    # run 
    consec_days.at[equation, aircraft] = LongestConseq(unique_arr= dates, days_legs= "days")
    consec_legs.at[equation, aircraft] = LongestConseq(unique_arr= legs, days_legs= "legs")
    max_intermittent.at[equation, aircraft] = max(intermitt)
    
    if total_occ_DF.at[equation, aircraft] >= MaxAllowedOccurrences \
    or consec_days.at[equation, aircraft] >= MaxAllowedConsecDays \
    or consec_legs.at[equation, aircraft] >= MaxAllowedConsecLegs \
    or max_intermittent.at[equation, aircraft] >= MaxAllowedIntermittent \
    or (legs >32600).any() \
    or (flags_jams == equation).any() \
    or ((flags_2in5 == equation).any() and check_2in5(dates)):
    
        if total_occ_DF.at[equation, aircraft] >= MaxAllowedOccurrences:
            flags_array.at[equation, aircraft] = flags_array.at[equation, aircraft] + "Total occurrences exceeded " + str(MaxAllowedOccurrences) + " occurrences. "
        
        if consec_days.at[equation, aircraft] >= MaxAllowedConsecDays:
            flags_array.at[equation, aircraft] = flags_array.at[equation, aircraft] + "Maximum consecutive days exceeded " + str(MaxAllowedConsecDays) + " days. "
            
        if consec_legs.at[equation, aircraft] >= MaxAllowedConsecLegs:
            flags_array.at[equation, aircraft] = flags_array.at[equation, aircraft] + "Maximum consecutive flight legs exceeded " + str(MaxAllowedConsecLegs) + " flight legs. "
            
        if max_intermittent.at[equation, aircraft] >= MaxAllowedIntermittent:
            flags_array.at[equation, aircraft] = flags_array.at[equation, aircraft] + "Maximum intermittent occurrences for one flight leg exceeded " + str(MaxAllowedIntermittent) + " occurrences. "

        if (legs >32600).any():
            flags_array.at[equation, aircraft] = flags_array.at[equation, aircraft] + "Flight legs above 32600. "

        if (flags_jams == equation).any():
            flags_array.at[equation, aircraft] = flags_array.at[equation, aircraft] + str(equation) + " occurred at least once. "

        if ((flags_2in5 == equation).any() and check_2in5(dates)): 
            flags_array.at[equation, aircraft] = flags_array.at[equation, aircraft] + str(equation) + " occurred at least twice in 5 days. "

        #populating the final array (Table)
        MAINtable_array_temp[0,0] = aircraft # SN
        MAINtable_array_temp[0,1] = MDCMessagesDF["EICAS"][MDCMessagesDF["Equation ID"] == equation].item() #EICAS
        MAINtable_array_temp[0,2] = MDCMessagesDF["Message"][MDCMessagesDF["Equation ID"] == equation].item() #Message
        MAINtable_array_temp[0,3] = MDCMessagesDF["LRU"][MDCMessagesDF["Equation ID"] == equation].item() #LRU
        MAINtable_array_temp[0,4] = MDCMessagesDF["ATA"][MDCMessagesDF["Equation ID"] == equation].item() #ATA
        MAINtable_array_temp[0,5] = equation #Eqn ID
        MAINtable_array_temp[0,6] = MDCMessagesDF["Message Type"][MDCMessagesDF["Equation ID"] == equation].item()
        MAINtable_array_temp[0,7] = MDCMessagesDF["Equation Description"][MDCMessagesDF["Equation ID"] == equation].item()
        MAINtable_array_temp[0,8] = total_occ_DF.at[equation, aircraft]
        MAINtable_array_temp[0,9] = consec_days.at[equation, aircraft]
        MAINtable_array_temp[0,10] = consec_legs.at[equation, aircraft]
        MAINtable_array_temp[0,11] = max_intermittent.at[equation, aircraft]
        MAINtable_array_temp[0,12] = dates.min()
        MAINtable_array_temp[0,13] = dates.max()
        MAINtable_array_temp[0,14] = str(flags_array.at[equation, aircraft])

        #if the input is empty set the priority to 4
        if MDCMessagesDF["Priority "][MDCMessagesDF["Equation ID"] == equation].item() == 0:
            MAINtable_array_temp[0,15] = 4
        else:
            MAINtable_array_temp[0,15] = MDCMessagesDF["Priority "][MDCMessagesDF["Equation ID"] == equation].item()

        #For B1-006424 & B1-006430 Could MDC Trend tool assign Priority 3 if logged on A/C below 10340, 15317. Priority 1 if logged on 10340, 15317, 19001 and up
        if equation == "B1-006424" or equation == "B1-006430":
            if int(aircraft) <= 10340 and int(aircraft) > 10000:
                MAINtable_array_temp[0,15] = 3
            elif int(aircraft) > 10340 and int(aircraft) < 11000:
                MAINtable_array_temp[0,15] = 1
            elif int(aircraft) <= 15317 and int(aircraft) > 15000:
                MAINtable_array_temp[0,15] = 3
            elif int(aircraft) > 15317 and int(aircraft) < 16000:
                MAINtable_array_temp[0,15] = 1
            elif int(aircraft) >= 19001 and int(aircraft) < 20000:
                MAINtable_array_temp[0,15] = 1

        #check content of "MEL or No-Dispatch"
        if MDCMessagesDF["MEL or No-Dispatch"][MDCMessagesDF["Equation ID"] == equation].item() == "0":
            MAINtable_array_temp[0,17] = ""
        else:
            MAINtable_array_temp[0,17] = MDCMessagesDF["MEL or No-Dispatch"][MDCMessagesDF["Equation ID"] == equation].item()

        #check content of "MHIRJ Input"
        if MDCMessagesDF["MHIRJ ISE inputs"][MDCMessagesDF["Equation ID"] == equation].item() == "0":
            MAINtable_array_temp[0,18] = ""
        else:
            MAINtable_array_temp[0,18] = MDCMessagesDF["MHIRJ ISE inputs"][MDCMessagesDF["Equation ID"] == equation].item()
        
        #check the content of MHIRJ ISE recommendation and add to array    
        if MDCMessagesDF["MHIRJ ISE Recommended Action"][MDCMessagesDF["Equation ID"] == equation].item() == "0":
            MAINtable_array_temp[0,19] = ""
        else:
            MAINtable_array_temp[0,19] = MDCMessagesDF["MHIRJ ISE Recommended Action"][MDCMessagesDF["Equation ID"] == equation].item()

        #check content of "additional"
        if MDCMessagesDF["Additional Comments"][MDCMessagesDF["Equation ID"] == equation].item() == "0":
            MAINtable_array_temp[0,20] = ""
        else:
            MAINtable_array_temp[0,20] = MDCMessagesDF["Additional Comments"][MDCMessagesDF["Equation ID"] == equation].item()

        #Check for the equation in the Top Messages sheet
        TopCounter = 0
        Top_LastRow = TopMessagesArray.shape[0]
        while TopCounter < Top_LastRow:

            #Look for the flagged equation in the Top Messages Sheet
            if equation == TopMessagesArray[TopCounter,4]:

                #Found the equation in the Top Messages Sheet. Put the information in the report
                MAINtable_array_temp[0,16] = "Known Nuissance: " + str(TopMessagesArray[TopCounter, 13]) \
                + " / In-Service Document: " + str(TopMessagesArray[TopCounter,11]) \
                + " / FIM Task: " + str(TopMessagesArray[TopCounter,10]) \
                + " / Remarks: " + str(TopMessagesArray[TopCounter,14])

                #Not need to keep looking
                TopCounter = TopMessagesArray.shape[0]

            else:
                #Not equal, go to next equation
                MAINtable_array_temp[0,16] = ""
                TopCounter += 1
        # End while

        if currentRow == 0:
            MAINtable_array = np.array(MAINtable_array_temp)      
        else:
            MAINtable_array = np.append(MAINtable_array,MAINtable_array_temp,axis=0)
        #End if Build MAINtable_array
        
        #Move to next Row on Main page for next flag
        currentRow = currentRow + 1
            
TitlesArrayHistory = ["AC SN", "EICAS Message", "MDC Message", "LRU", "ATA", "B1-Equation", "Type",
               "Equation Description", "Total Occurrences", "Consecutive Days", "Consecutive FL",
              "Intermittent", "Date from", "Date to", "Reason(s) for flag", "Priority", "Known Top Message - Recommended Documents", "MEL or No-Dispatch", 
              "MHIRJ Input", "MHIRJ Recommendation", "Additional Comments"]

# Converts the Numpy Array to Dataframe to manipulate
#pd.set_option('display.max_rows', None)
# Main table

OutputTableHistory = pd.DataFrame(data= MAINtable_array, columns= TitlesArrayHistory).fillna(" ")
OutputTableHistory = OutputTableHistory.merge(AircraftTailPairDF, on= "AC SN") # Tail# added to the end (last column)
OutputTableHistory = OutputTableHistory[["Tail#", "AC SN", "EICAS Message", "MDC Message", "LRU", "ATA", "B1-Equation", "Type",
               "Equation Description", "Total Occurrences", "Consecutive Days", "Consecutive FL",
              "Intermittent", "Date from", "Date to", "Reason(s) for flag", "Priority", "Known Top Message - Recommended Documents","MEL or No-Dispatch",
              "MHIRJ Input", "MHIRJ Recommendation", "Additional Comments"]].sort_values(by= ["Type", "Priority"]) # Tail# added to output table which means that column order has to be re orderedb8632868 2076


Run cell #10 to see a preview of the history analysis

In [15]:
# 10
# to toggle the output in this cell (or any cell) press "esc + o" - find more shortcuts using "esc + h"
# Check if user desires excel output

# In order to properly show the color, the "style" method seen below will change the Dataframe data type to a "Styler"
# data type. which will not allow other actions to work (i.e. drop, apply, etc) since those methods dont exist for Styler
# data type. a copy of the output table is created to display and rewrite that table everytime the cell is run. 
copy_OutputTableHistory = OutputTableHistory.copy(deep= True)
copy_OutputTableHistory = copy_OutputTableHistory.style.apply(highlightJams, axis=1)


excelcheck  = input("Do you want to print this report to excel? yes/no: ")
if excelcheck.lower() != "yes" and excelcheck.lower() != "no":
    print("Only yes or no allowed")
elif excelcheck.lower() == "no":
    display(copy_OutputTableHistory)
elif excelcheck.lower() == "yes":
    copy_OutputTableHistory.to_excel(input("Input the desired filename, '.xlsx' is added automatically: ") + ".xlsx", index= False, header= True)
    display(copy_OutputTableHistory)


,Tail#,AC SN,EICAS Message,MDC Message,LRU,ATA,B1-Equation,Type,Equation Description,Total Occurrences,Consecutive Days,Consecutive FL,Intermittent,Date from,Date to,Reason(s) for flag,Priority,Known Top Message - Recommended Documents,MEL or No-Dispatch,MHIRJ Input,MHIRJ Recommendation,Additional Comments
0,755EV,10185,L BLEED LOOP (C) / R BLEED LOOP (C),LEAK DET CH A&B FAULT,AILC,30-00,B1-005945,Fault Message,Both leak detection channels arereported failed by arinc busses toDCUs,3.000000,2.000000,3.000000,0.000000,2021-08-29,2021-08-30,Total occurrences exceeded 2 occurrences. Maximum consecutive days exceeded 2 days. Maximum consecutive flight legs exceeded 2 flight legs.,1,Known Nuissance: N / In-Service Document: None / FIM Task: 30−10−00−810−804 / Remarks: Cross check with EICAS,36-21-06,AILC both channel fault. Do operational test if message is still posted replace AILC. (Same FIM as B1-005946/47),Follow FIM TASK 30−10−00−810−804: – AILC A105 unserviceable.,FIM is good
1,724SK,10189,L BLEED LOOP (C) / R BLEED LOOP (C),LEAK DET CH A&B FAULT,AILC,30-00,B1-005945,Fault Message,Both leak detection channels arereported failed by arinc busses toDCUs,3.000000,1.000000,3.000000,0.000000,2021-09-03,2021-09-03,Total occurrences exceeded 2 occurrences. Maximum consecutive flight legs exceeded 2 flight legs.,1,Known Nuissance: N / In-Service Document: None / FIM Task: 30−10−00−810−804 / Remarks: Cross check with EICAS,36-21-06,AILC both channel fault. Do operational test if message is still posted replace AILC. (Same FIM as B1-005946/47),Follow FIM TASK 30−10−00−810−804: – AILC A105 unserviceable.,FIM is good
5,724SK,10189,L BLEED DUCT (W) or DUCT MON FAULT (S) if 1 loop,LEAK-RUN EVENT LOC TEST,L BLEED LOOP A,36-00,B1-349536,Fault Message,Bleed air leak detected. RunEvent Location Test to determinelocation of the leak.,4.000000,3.000000,1.000000,0.000000,2021-08-28,2021-08-30,Total occurrences exceeded 2 occurrences. Maximum consecutive days exceeded 2 days.,1,"Known Nuissance: N / In-Service Document: CRJ700/900-SL-36-017 / FIM Task: 36−21−01−810−813 / Remarks: New AILC upgrade / Troubleshooting tips Service Letter cancelled, included in FIM lncludes every MDC B1-code associated to DUCT MON FAULT (S) (open and short circuits), L/R BLEED DUCT (W), ANTI-ICE DUCT (W), L/R COWL A/I DUCT (W)",(W) No Dispatch (S) 30-12-06 / 36-21-06,MDT logic: AILC L352b11 set (Ch. A only) Top #43 in fleet Top 50 MDC msgs See SL-36-018: There is an error in the Event Location of the AILC. This information is accessible through the MDC. This letter also gives the values for the corrected Event Location. This SL is now in the FIM. Follow FIM.,Follow FIM TASK 36−21−01−810−813: 1. A component of the left side bleed−air distribution system is unserviceable and causes a leak. 2. A bleed−air leak sensing element is not installed correctly (too close to the hot bleed air ducting or any other hot components such as valves and structure),There is a SL CRJ700/705/900/1000-SL-36-018. To inform operators of an error in the Event Location of the Anti-Ice Leak Detection Controller. This information is accessible through the MDC. This letter also gives the values for the corrected Event Location. This SL is now in the FIM. Follow FIM.
6,790SK,10292,L BLEED LOOP (C) / R BLEED LOOP (C),LEAK DET CH A&B FAULT,AILC,30-00,B1-005945,Fault Message,Both leak detection channels arereported failed by arinc busses toDCUs,4.000000,1.000000,4.000000,0.000000,2021-09-03,2021-09-03,Total occurrences exceeded 2 occurrences. Maximum consecutive flight legs exceeded 2 flight legs.,1,Known Nuissance: N / In-Service Document: None / FIM Task: 30−10−00−810−804 / Remarks: Cross check with EICAS,36-21-06,AILC both channel fault. Do operational test if message is still posted replace AILC. (Same FIM as B1-005946/47),Follow FIM TASK 30−10−00−810−804: – AILC A105 unserviceable.,FIM is good
7,790SK,10292,SPLR/STAB FAULT (S),FAN 2 FAILED,SSCU 1,27-60,B1-008014,Fault Message,SSCU 1 fan is failed.,2

# If there is a Jam related flag, the supporting info will show up here

Run the cell below to obtain the other messages posted on the same flight leg as the Jam Message.
Input the Aircraft SN: 1XXXX to obtain the messages related to the Jam.

Example of process would be:
- Run History Report
- Look at the highlighted flags, choose one of those Aircraft SN and copy
- Paste it into the input box below
- You will obtain the list of messages that also occurred on the same flight leg
- You can also choose to print that table to excel in the next cell


In [ ]:
ACchosen = input("From your analysis, what Aircraft SN do you want to look at?: ")

RelatedtoFlagged = flagsinreport(OutputTable= OutputTableHistory, Aircraft= ACchosen)
RelatedtoFlagged

In [ ]:
RelatedtoFlagged.to_excel("") # write the desired file name in the quotes

# Flags report

This section creates a report and then is able to print to excel

Make sure you run the cell above # 11

Choose an aircraft from the **history** report (e.g. 15400)

Choose a B1 Message also from the **history** report 

Choose whether you want a new report or not (i.e. **True** or **False**)

A key issue is that after you create a new report, you must go back to cell # 12 and change *Newreport* to **False** before continuing to produce the report, otherwise you will overwrite the line you just added

Run the cell # 13 to add a new line to the report and display the updated report

Sample routine for a new report: 

1. cell #12: Choose ACSN and Bcode, Newreport=True -> run cell
2. cell #13: Run
3. cell #12: Choose different ACSN and Bcode, Newreport=False -> run cell
4. cell #13: Run
5. Repeat steps 3 and 4 until report is done
6. cell #14: Run to print to excel. Choose new worksheet name

In [ ]:
# 11
# create a flags report 
Flagsreport = 1 # this is here to initialize the variable. user must start report by choosing Newreport = True
def Toreport(Flagsreport, AircraftSN, Bcode, newreport= False, HistoryReport= OutputTableHistory):
    '''Populates a report with input from the previous report, aircraft serial number and B1 message code'''
    # if the user wants a brand new report
    if newreport:
        del Flagsreport
        Flagsreport = pd.DataFrame(data= None)
    indexedreport = HistoryReport.set_index(["AC SN", "B1-Equation"])
    
    #creating dataframe to look at dates
    if CurrentFlightPhaseEnabled == 1: #Show all, current and history
        DatesDF = MDCdataDF[["DateAndTime","Equation ID", "Aircraft"]].copy()

    elif CurrentFlightPhaseEnabled == 0: #Only show history
        DatesDF = MDCdataDF[["DateAndTime","Equation ID", "Aircraft", "Flight Phase"]].copy()
        DatesDF = DatesDF.replace(False, np.nan).dropna(axis=0, how='any')
        DatesDF = DatesDF[["DateAndTime","Equation ID", "Aircraft"]].copy()
        
    # this exists to check which dates are present for the specific aircraft and message chosen
    counts = pd.DataFrame(data= DatesDF.groupby(['Aircraft', "Equation ID", "DateAndTime"]).agg(len), columns= ["Counts"])
    counts
    DatesfoundinMDCdata = counts.loc[(AircraftSN, Bcode)].resample('D')["Counts"].sum().index
    
    #create the new row that will be appended to the existing report
    newrow = indexedreport.loc[(AircraftSN, Bcode), ["Tail#", "ATA", "LRU", "MDC Message", "Type","EICAS Message", "MEL or No-Dispatch", "MHIRJ Input", "MHIRJ Recommendation", "Additional Comments"]].to_frame().transpose()
    newrow.insert(loc= 0, column= "AC SN", value= AircraftSN)
    newrow.insert(loc= 3, column= "B1-code", value= Bcode)
    newrow.insert(loc= 8, column= "Date From", value= DatesfoundinMDCdata.min().date()) #.date()removes the time data from datetime format
    newrow.insert(loc= 9, column= "Date To", value= DatesfoundinMDCdata.max().date())
    newrow.insert(loc= 10, column= "SKW action WIP", value= "")
    newrow = newrow.rename(columns= {"AC SN":"MSN", "MDC Message": "Message", "EICAS Message":"Potential FDE"})
    
    # append the new row to the existing report
    Flagsreport = Flagsreport.append(newrow, ignore_index= True)
    
    return Flagsreport

In [ ]:
# 12
ACSN = input("Choose the AC serial number you want to add: ")
bCode1 = b1Format()
QNewreport = input("Do you want a new report? yes/no: ")
if QNewreport == "yes":
    Newreport = True
elif QNewreport == "no":
    Newreport = False

In [ ]:
# 13
from IPython.display import display, HTML

def pretty_print(df):
    '''Here to add a new line to the displayed report'''
    return display( HTML( df.to_html().replace("\\n","<br>") ) )

Flagsreport = Toreport(Flagsreport, AircraftSN= ACSN, Bcode= bCode1, newreport= Newreport)
pretty_print(Flagsreport)

Run cell # 14 to print the report to a new excel spreadsheet

In [ ]:
# 14
Flagsreport.to_excel("") # write the desired file name in the quotes

# Charts

Use this section based on the results of the analysis 

Choose an aircraft to analyze (e.g. 15400)

Choose a B1 Message, ATA to analyze

### Chart 1 

analyzes the occurrence of B1-messages for the chosen aircraft

#### Inputs: 

In [ ]:
# 15

AircraftToStudy = input("Choose the AC serial number you want to analize: ") # in this input input the AC SN (e.g "15380", "10307")
Topvalues = 15 # Top number of values you want to see in the graph - 5? 10? 25?

#### Output: Run cell below

In [ ]:
# 16


# include the current flight legs
selection = MDCdataDF[["Equation ID", "Aircraft"]].copy()
total_occ_DF = selection.value_counts().unstack()
    
# B1 Messages Occurrence per Aircraft    
messagestoshow = total_occ_DF[AircraftToStudy].sort_values().dropna().tail(Topvalues).index.to_frame(index= False, name = "Message")
messagesdescriptionDF = MDCMessagesDF[["Equation ID", "Message", "EICAS", "LRU", "ATA"]].set_index(["Equation ID"])

def Definelabels(messages, Messagedata= messagesdescriptionDF):
    '''Input of dataframe containing Bcode messages to be matched and joined with their respective descriptions'''
    for i in range(len(messages)):
        Bcode = messages.at[i, "Message"]
        LRUdata = Messagedata.loc[Bcode, "LRU"]
        ATAdata = Messagedata.loc[Bcode, "ATA"]
        messages.at[i, "Message"] = str(messages.at[i, "Message"] + "\n" + LRUdata + "\n" + ATAdata)
    return messages["Message"].to_list()

Plottinglabels = Definelabels(messagestoshow)
Plottingarray = total_occ_DF[AircraftToStudy].sort_values().dropna().tail(Topvalues).values

#Image settings
spacing = np.arange(start= 0, stop= len(Plottinglabels)*2, step= 2) # label locations
height = 0.9 # bar height
fig1, ax1 = plt.subplots(figsize= (16,9)) # objects for figure and axes, figsize controls the size of the output window
rects1 = ax1.barh(spacing, Plottingarray, height= height, label= AircraftToStudy)

# Add some text for labels, title and custom x-axis tick labels, etc.
ax1.set_xlabel('Total Number of messages')
tail = AircraftTailPairDF.loc[AircraftTailPairDF["AC SN"] == AircraftToStudy].iloc[0].at["Tail#"]
ax1.set_title('Occurrence of Messages in ' + AircraftToStudy + " / " + tail)
ax1.set_yticks(spacing)
ax1.set_yticklabels(Plottinglabels)
ax1.grid(b= True, alpha= 0.3)
ax1.legend()

def autolabel(rects): # https://matplotlib.org/examples/api/barchart_demo.html
    """
    Attach a text label above each bar displaying its height
    """
    for rect in rects:
        width = rect.get_width()
        ax1.text(width, rect.get_y() + height/2, # position in graph
                '%d' % int(width), # text to be shown
                ha='left') #hor alignment

autolabel(rects1) 

fig1.tight_layout()

plt.show()

If you want the information on a specific message, change the value between the quotes next to _messagecode_ and run the cell below

In [ ]:
# 17
# Input here the message you desire to explore
# it will provide the Message Description, EICAS alert, LRU and ATA

bCode2 = b1Format()

messagesdescriptionDF = MDCMessagesDF[["Equation ID", "Message", "EICAS", "LRU", "ATA"]].set_index(["Equation ID"])
messagesdescriptionDF.loc[bCode2]

## Chart 2 

Analyzes the occurrence of the chosen ATA and shows the occurrence in aircraft

You can choose how many aircraft are shown in the chart by changing the value for _Topvalues2_  

#### Inputs:

In [ ]:
# 18

ATAtoStudy = input("Choose the ATA you want to analize: ") # in this "String" input the 2 dig or 4 dig ATA to study (e.g "32-61", "21-61" or "21", "32"),
                    # Remember the 4 dig ATA has the dash in between
Topvalues2 = 10 # Top number of values you want to see in the graph - 5? 10? 25?

#### Outputs: Run cell below

In [ ]:
# 19

chart2DF = pd.merge(left = MDCdataDF[["Aircraft","ATA Main", "ATA"]], right = AircraftTailPairDF, left_on="Aircraft", right_on="AC SN")
chart2DF["Aircraft"] = chart2DF["Aircraft"] + " / " + chart2DF["Tail#"]
chart2DF.drop(labels = ["AC SN", "Tail#"], axis = 1, inplace = True)

# check length of variable ATAtoStudy 
if len(ATAtoStudy) == 2:
    # Convert 2 Dig ATA array to Dataframe to analyze
    TwoDigATA_DF = chart2DF.drop("ATA", axis = 1).copy()
    # Count the occurrence of each ata in each aircraft
    ATAOccurrenceDF = TwoDigATA_DF.value_counts().unstack()
    
    Plottinglabels = ATAOccurrenceDF[int(ATAtoStudy)].sort_values().dropna().tail(Topvalues2).index.to_list() # Aircraft Labels
    Plottingarray = ATAOccurrenceDF[int(ATAtoStudy)].sort_values().dropna().tail(Topvalues2).values # values for the plotting
    
elif len(ATAtoStudy) == 5:
    # Convert 4 Dig ATA array to Dataframe to analyze
    FourDigATA_DF = chart2DF.drop("ATA Main", axis = 1).copy()
    # Count the occurrence of each ata in each aircraft
    ATAOccurrenceDF = FourDigATA_DF.value_counts().unstack()
    
    Plottinglabels = ATAOccurrenceDF[ATAtoStudy].sort_values().dropna().tail(Topvalues2).index.to_list() # Aircraft Labels
    Plottingarray = ATAOccurrenceDF[ATAtoStudy].sort_values().dropna().tail(Topvalues2).values # values for the plotting

#Image settings
spacing = np.arange(len(Plottinglabels)) # label locations
height = 0.35 # bar height
fig2, ax2 = plt.subplots()
rects2 = ax2.barh(spacing, Plottingarray, height= height, label= ATAtoStudy)

# Add some text for labels, title and custom x-axis tick labels, etc.
ax2.set_xlabel('Occurrences for ATA ' + ATAtoStudy + ' for each AC')
ax2.set_title('ATA ' + ATAtoStudy + ' Occurrence in each AC')
ax2.set_yticks(spacing)
ax2.set_yticklabels(Plottinglabels)
ax2.grid(b= True, alpha= 0.3)
ax2.legend()

def autolabel(rects): # https://matplotlib.org/examples/api/barchart_demo.html
    """
    Attach a text label above each bar displaying its height
    """
    for rect in rects:
        width = rect.get_width()
        ax2.text(width, rect.get_y() + height/2, # position in graph
                '%d' % int(width), # text to be shown
                ha='left') #hor alignment

autolabel(rects2)        
fig2.tight_layout()

plt.show()
# search for occurring messages in that ATA in results
# display flags encountered for each of those messages

## Chart 3

Shows the trend of the chosen message with the occurrences per day for a chosen aircraft

Choose the aircraft and B1 message

#### Inputs:

In [ ]:
# 22

ACtostudy2 = input("Choose the AC serial number you want to analize: ") # input the AC you want to look at
Messagetostudy2 = b1Format()

#### Outputs Run cell below

In [ ]:
# 23

def datefill(datesindata, valuesindata, NumDays, enddate):
    '''Creates an array with the values found in data and fills the rest of the date range with zeros'''
    Datatoplot = pd.date_range(end= enddate, periods= NumDays, freq= "D")
    Datatoplot = Datatoplot.to_frame(index= False, name = "Date")
    Datatoplot["Values"] = 0
    Datesindata = datesindata.to_frame(index= False, name = "Date")
    
    for i in range(NumDays):
            hi = Datatoplot.at[i, "Date"]
            for j in range(len(Datesindata)):
                bye = Datesindata.at[j, "Date"]
                
                if hi == bye:
                    Datatoplot.at[i, "Values"] = valuesindata[j]
                    
    Matplotlibdates = matplotlib.dates.date2num(Datatoplot.Date)
    Valuestoplot = Datatoplot.Values
    return Matplotlibdates, Valuestoplot


#creating dataframe to look at dates
if CurrentFlightPhaseEnabled == 1: #Show all, current and history
    DateNmessageDF = MDCdataDF[["DateAndTime","Equation ID", "Aircraft"]].copy()
    
elif CurrentFlightPhaseEnabled == 0: #Only show history
    DateNmessageDF = MDCdataDF[["DateAndTime","Equation ID", "Aircraft", "Flight Phase"]].copy()
    DateNmessageDF = DateNmessageDF.replace(False, np.nan).dropna(axis=0, how='any')
    DateNmessageDF = DateNmessageDF[["DateAndTime","Equation ID", "Aircraft"]].copy()
    
pd.to_datetime(DateNmessageDF["DateAndTime"])
counts = pd.DataFrame(data= DateNmessageDF.groupby(['Aircraft', "Equation ID", "DateAndTime"]).agg(len), columns= ["Counts"])

# formatting the dates, only changed "minticks" to fix issue with plot autofilling date values between datapoints
xtick_locator = matplotlib.dates.AutoDateLocator(minticks = 1, maxticks=None)
xtick_formatter = matplotlib.dates.AutoDateFormatter(xtick_locator)

# data to pass into function "datefill" to format
DatesfoundinMDCdata = counts.loc[(ACtostudy2, Messagetostudy2)].resample('D')["Counts"].sum().index
valuesfoundinMDCdata = counts.loc[(ACtostudy2, Messagetostudy2)].resample('D')["Counts"].sum()

#image axes
xaxis, yaxis= datefill(DatesfoundinMDCdata, valuesfoundinMDCdata, NumberofDays, latestDay)    

#image settings .to_pydatetime()
fig4, ax4 = plt.subplots()
rects4 = ax4.plot_date(xaxis, yaxis, xdate= True, linestyle= "-")

#plot format
ax4.set_xlabel('Occurrence dates')
ax4.set_ylabel("Occurrence per day")
ax4.set_ylim(ymin=0)
tail = AircraftTailPairDF.loc[AircraftTailPairDF["AC SN"] == ACtostudy2].iloc[0].at["Tail#"]
ax4.set_title('Date trend for ' + ACtostudy2 + " / " + tail + ' and message ' + Messagetostudy2)
ax4.grid(b= True, which= "both", alpha= 0.5, axis= "both")
ax4.xaxis.set_tick_params(rotation=30, labelsize=10)
ax4.xaxis.set_major_locator(xtick_locator)
ax4.xaxis.set_major_formatter(xtick_formatter)


fig4.tight_layout()

plt.show()

## Chart 4

Ranks the ATAs in the Chosen Report to see which ATAs are more present in the timeframe

You can change the number of ATAs that pop up on the chart using the variable _Topcount_

**Inputs**

In [ ]:
# 24

ReportChosen = input("Choose the report you want to analize: History/Daily ") # "History" or "Daily"
Topcount = 10

**Outputs run cell below**

In [ ]:
# 25

# Chart 4 - gets the top ATAs in the alerts 
if ReportChosen.lower() == "history":
    Dataforchart = OutputTableHistory["ATA"]

elif ReportChosen.lower() == "daily":
    Dataforchart = OutputTableDaily["ATA"]

    
labels = Dataforchart.value_counts().index
countsofATAs = Dataforchart.value_counts().sort_values().tail(Topcount)

#Image settings
spacing = np.arange(len(countsofATAs)) # label locations
height = 0.35 # bar height
fig5, ax5 = plt.subplots()
rects5 = ax5.barh(spacing, countsofATAs, height= height, label= "ATAs in the " + ReportChosen + " Report")

# Add some text for labels, title and custom x-axis tick labels, etc.
ax5.set_xlabel('Occurrences for ATAs in the Report')
ax5.set_title('ATA Occurrence in the ' + ReportChosen + ' report')
ax5.set_yticks(spacing)
ax5.set_yticklabels(countsofATAs.index)
ax5.grid(b= True, alpha= 0.3)
ax5.legend()

def autolabel(rects): # https://matplotlib.org/examples/api/barchart_demo.html
    """
    Attach a text label above each bar displaying its height
    """
    for rect in rects:
        width = rect.get_width()
        ax5.text(width, rect.get_y() + height/2, # position in graph
                '%d' % int(width), # text to be shown
                ha='left') #hor alignment

autolabel(rects5)        
fig5.tight_layout()

plt.show()

## Chart 5

Shows the trend of the chosen message with the intermittence per flight leg for chosen aircraft (1XXXX)

Choose the Aircraft and B1 message 

**Inputs**

In [ ]:
# 26

ACtostudy3 = input("Choose the AC serial number you want to analyze: ") # input the AC you want to look at
Messagetostudy3 = b1Format()

**Outputs run cell below**

In [ ]:
# 27

#creating dataframe to look at dates
if CurrentFlightPhaseEnabled == 1: #Show all, current and history
    FLintermittentDF = MDCdataDF[["Flight Leg No","Equation ID", "Aircraft", "Intermittent"]].copy()
    
elif CurrentFlightPhaseEnabled == 0: #Only show history
    FLintermittentDF = MDCdataDF[[ "Flight Leg No", "Equation ID", "Aircraft", "Intermittent"]].copy()
    FLintermittentDF = FLintermittentDF.replace(0, np.nan).dropna(axis=0, how='any')
    FLintermittentDF = FLintermittentDF[["Flight Leg No","Equation ID", "Aircraft", "Intermittent"]].copy()
    
FLintermittentDF = FLintermittentDF.set_index(["Aircraft", "Equation ID"]).sort_index().loc[(ACtostudy3, Messagetostudy3)]

def flightlegfill(FLintermittentDF):
    '''Creates an array with the values found in data and fills the rest of the flight leg range with zeros'''
    minFL = int(FLintermittentDF.min().iloc[0])
    maxFL = int(FLintermittentDF.max().iloc[0])
    Flightlegstoplot = pd.Series(range(minFL, maxFL+1))
    Flightlegstoplot = Flightlegstoplot.to_frame(name= "FL No")
    Flightlegstoplot["Values"] = 0
    FLintermittentDF = FLintermittentDF.reset_index()
    
    for i in range(len(Flightlegstoplot)):
            hello = Flightlegstoplot.at[i, "FL No"]
            for j in range(len(FLintermittentDF)):
                there = FLintermittentDF.at[j, "Flight Leg No"]
                
                if hello == there:
                    Flightlegstoplot.at[i, "Values"] = FLintermittentDF.at[j, "Intermittent"]
                    
    Flightlegs = Flightlegstoplot["FL No"]
    Valuestoplot = Flightlegstoplot.Values
    return Flightlegs, Valuestoplot

#image axes
xaxis, yaxis = flightlegfill(FLintermittentDF)

#image settings
fig6, ax6 = plt.subplots()
rects6 = ax6.plot(xaxis, yaxis, marker= "o", linestyle= "-")

#plot format
ax6.set_xlabel('Flight Leg')
ax6.set_ylabel("Intermittence")
ax6.set_ylim(ymin=0)
ax6.xaxis.set_major_locator(matplotlib.ticker.MaxNLocator(nbins= len(xaxis)*2, integer= True))
tail = AircraftTailPairDF.loc[AircraftTailPairDF["AC SN"] == ACtostudy3].iloc[0].at["Tail#"]
ax6.set_title('Intermittence trend for ' + ACtostudy3 + " / " + tail + ' and message ' + Messagetostudy3)
ax6.grid(b= True, which= "both", alpha= 0.5, axis= "both")
ax6.xaxis.set_tick_params(rotation=30, labelsize=10)

fig6.tight_layout()

plt.show()

# History Report Delta Highlighting

Add the path of the two reports you're comparing between the quotes (" ") for *prev_history_path* & *curr_history_path*

Run cells 28 and 29 to obtain the delta report

You will be prompted for a name for the delta report

In [43]:
# 28
prev_history_path = r"/Users/kanu/Desktop/MHIRJ/19-21.xlsx"
curr_history_path = r"/Users/kanu/Desktop/MHIRJ/20-22.xlsx"

In [44]:
# 29
# define DataFrames of reports
prev_history = pd.read_excel(prev_history_path)
curr_history = pd.read_excel(curr_history_path)

# define lists, does the message in the new report exist in the previous or not
True_list, False_list = create_delta_lists(prev_history, curr_history)

curr_history.set_index(["AC SN", "B1-Equation"], drop= False, inplace= True)
prev_history.set_index(["AC SN", "B1-Equation"], drop= False, inplace= True)
# grab only what exists in the new report from the old report
prev_history_nums = prev_history.loc[True_list]
# add the items that only exist in the new report and add them to the old report 
# (this has to be after the True/False Lists are created, bc if done before then everything will be True (list))
# this was done bc the slicing done for slice_ needs to compare two dataframes with identical indexes
prev_history_nums = prev_history_nums.append(curr_history.loc[False_list])
# sort indexes
curr_history.sort_index(inplace= True)
prev_history_nums.sort_index(inplace= True)

print("prev_history : ",prev_history.columns)
print("curr_history : ",curr_history.columns)
# color what existed previously
delta = curr_history.style.apply(highlight_delta, axis= 1, delta_list= True_list)
# color what is new
delta = delta.apply(highlight_delta, axis= 1, delta_list= False_list)

# coloring the counters
idx = pd.IndexSlice
# comparing the counters on each report 
# since there are some rows that are added to the prev_history_nums (see above), 
# the values on both dataframes will be equal at the corresponding indexes and wont be highlighted 
# the logic here will only highlight whats strictly greater
slice_ = idx[idx[curr_history["Total Occurrences"] > prev_history_nums["Total Occurrences"]], ["Total Occurrences"]]
slice_2 = idx[idx[curr_history["Consecutive Days"] > prev_history_nums["Consecutive Days"]], ["Consecutive Days"]]
slice_3 = idx[idx[curr_history["Consecutive FL"] > prev_history_nums["Consecutive FL"]], ["Consecutive FL"]]
slice_4 = idx[idx[curr_history["Intermittent"] > prev_history_nums["Intermittent"]], ["Intermittent"]]
delta = delta.set_properties(**{'background-color': '#fabf8f'}, subset=slice_)
delta = delta.set_properties(**{'background-color': '#fabf8f'}, subset=slice_2)
delta = delta.set_properties(**{'background-color': '#fabf8f'}, subset=slice_3)
delta = delta.set_properties(**{'background-color': '#fabf8f'}, subset=slice_4)
# printing to excel
delta.to_excel(input("Input the desired filename, '.xlsx' is added automatically: ") + ".xlsx", index= False)

FileNotFoundError: [Errno 2] No such file or directory: '/Users/kanu/Desktop/MHIRJ/20-21.xlsx'